<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/Vision_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import random
from google.colab import drive, files
import datetime
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback, TensorBoard
import tensorflow.keras.mixed_precision as mixed_precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import math  # For mathematical functions
import PIL  # Pillow library for image processing
import PIL.Image  # Specific import from Pillow for image manipulation
import sys  # Provides access to system-specific parameters and functions
from google.colab import files  # For file handling in Google Colab
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/markbilszky/agile-manufacturing-tdk
License(s): unknown
 99% 1.88G/1.89G [00:26<00:00, 92.1MB/s]
100% 1.89G/1.89G [00:26<00:00, 76.9MB/s]
Dataset downloaded successfully.


In [3]:
# Set your paths here
base_path = './'  # Replace with the directory containing Reference, error_2, and error_3
new_folder_path = os.path.join(base_path, 'training_data')

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# Move the folders
folders_to_move = ['Reference', 'error_2', 'error_3']
for folder_name in folders_to_move:
    shutil.move(os.path.join(base_path, folder_name), new_folder_path)

print("Folders moved successfully!")

Folders moved successfully!


In [4]:
# Path to the training data directory
data_dir = './training_data'

# List of main folders to check for nested duplicates
main_folders = os.listdir(data_dir)

for folder in main_folders:
    folder_path = os.path.join(data_dir, folder)

    # Check if this path is indeed a folder
    if os.path.isdir(folder_path):
        # Path to the nested duplicate folder (if exists)
        nested_folder_path = os.path.join(folder_path, folder)

        # Check if a nested folder with the same name exists
        if os.path.isdir(nested_folder_path):
            print(f"Found duplicate folder: {nested_folder_path}")

            # Move all files from nested folder to the main folder
            for filename in os.listdir(nested_folder_path):
                file_path = os.path.join(nested_folder_path, filename)
                target_path = os.path.join(folder_path, filename)

                # Move file to the main folder
                if os.path.isfile(file_path):
                    shutil.move(file_path, target_path)
                    #print(f"Moved {file_path} to {target_path}")

            # Remove the nested duplicate folder after moving its contents
            os.rmdir(nested_folder_path)
            print(f"Removed duplicate folder: {nested_folder_path}")

print("Duplicate folders cleaned up successfully.")

Found duplicate folder: ./training_data/error_2/error_2
Removed duplicate folder: ./training_data/error_2/error_2
Found duplicate folder: ./training_data/Reference/Reference
Removed duplicate folder: ./training_data/Reference/Reference
Found duplicate folder: ./training_data/error_3/error_3
Removed duplicate folder: ./training_data/error_3/error_3
Duplicate folders cleaned up successfully.


In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [6]:
data_dir = './training_data'  # Ensure this path is correct for your dataset
all_classes = sorted(os.listdir(data_dir))  # List and sort class folders in the dataset directory
train_files = []


for cls in all_classes:
    cls_dir = os.path.join(data_dir, cls)
    if os.path.isdir(cls_dir):
        files = [os.path.join(cls_dir, file) for file in os.listdir(cls_dir) if file.endswith(('png', 'jpg', 'jpeg'))]
        print(f"Class: {cls}, Number of files: {len(files)}")
        train_files.extend(files)


Class: Reference, Number of files: 1000
Class: error_2, Number of files: 7903
Class: error_3, Number of files: 10000


In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Hyperparameters for the learning rate
fixed_lr = 0.002  # Set a fixed learning rate
total_epochs = 1000
batch_size = 8
img_height, img_width = 140, 190
data_dir = './training_data'

# Split dataset into training, validation, and test sets
all_classes = sorted(os.listdir(data_dir))
train_files = []
val_files = []

# Collect all file paths and their corresponding classes
for cls in all_classes:
    cls_dir = os.path.join(data_dir, cls)
    if os.path.isdir(cls_dir):
        files = [os.path.join(cls_dir, file) for file in os.listdir(cls_dir) if file.endswith(('png', 'jpg', 'jpeg'))]
        train_files.extend(files)

# Create a split for training, validation, and test data
train_files, test_files = train_test_split(train_files, test_size=0.15, random_state=42)  # 20% for test
train_files, val_files = train_test_split(train_files, test_size=0.15, random_state=42)  # 20% of the remaining 80% for validation

# Function to create a generator from file paths
def create_generator(file_paths, batch_size):
    while True:
        np.random.shuffle(file_paths)
        for i in range(0, len(file_paths), batch_size):
            batch_paths = file_paths[i:i+batch_size]
            images = []
            labels = []
            for path in batch_paths:
                img = tf.keras.preprocessing.image.load_img(path, target_size=(img_height, img_width))
                img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Rescale
                images.append(img)
                label = path.split(os.path.sep)[-2]  # Get the label from the directory name
                labels.append(all_classes.index(label))
            yield np.array(images), tf.keras.utils.to_categorical(labels, num_classes=len(all_classes))

# Create the data generators
train_generator = create_generator(train_files, batch_size=batch_size)
val_generator = create_generator(val_files, batch_size=batch_size)
test_generator = create_generator(test_files, batch_size=batch_size)

# Set up TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

"""
# Set mixed precision policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
"""
# Define the Vision Transformer model
def create_vit_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(8, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer='l2')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # Reshape for transformer input
    x = layers.Reshape((-1, x.shape[-1]))(x)

    # Transformer block
    for _ in range(2):  # Number of transformer blocks
        x_res = x
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=2, key_dim=8)(x, x)
        x = layers.Add()([x_res, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x_res = x
        x = layers.Dense(64, activation='relu', kernel_regularizer='l2')(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(x_res.shape[-1])(x)
        x = layers.Add()([x_res, x])

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer='l2')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_vit_model((img_height, img_width, 3))  # Input shape should have 1 channel for grayscale
model.compile(optimizer=Adam(learning_rate=fixed_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=2
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

# Calculate the number of steps per epoch
steps_per_epoch = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size
test_steps = len(test_files) // batch_size

# Train the model with the sinusoidal learning rate callback
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=total_epochs,
    callbacks=[early_stopping, model_checkpoint, tensorboard_callback]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=test_steps)
print(f'Test Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/1000
   5/1707 ━━━━━━━━━━━━━━━━━━━━ 25:17:35 53s/step - accuracy: 0.2612 - loss: 1.8262

In [ ]:
model.summary()

In [ ]:
print('Rétegek:\n')
for layer in model.layers:
    print("Réteg neve: ", layer.name, ', tanítható: ', layer.trainable)
    print(layer.get_config(),'\n')

In [ ]:
!pip3 install keras-visualizer
from keras_visualizer import visualizer
import matplotlib.image as mpimg
visualizer(model, file_format='png', view=True)
img = mpimg.imread('graph.png')
fig = plt.figure(figsize=[20,9])
plt.imshow(img)
plt.axis('off')

In [ ]:
# Logging images to TensorBoard
# Logging images to TensorBoard
file_writer = tf.summary.create_file_writer(logdir)

with tf.name_scope("Test") as scope:
    with file_writer.as_default():
        # Reshape the first three images from your test dataset
        img = np.reshape(test_images[0:3], (-1, img_height, img_width, 3))  # Adjust to (num_samples, height, width, channels)
        tf.summary.image("Test Images", img, step=0, max_outputs=3, description="Sample images from the test set")

# Remember to flush the writer to ensure the images are saved
file_writer.flush()

In [ ]:
# Get predictions from the model
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)  # Get the predicted class labels

# Get the true class labels from the test set
true_classes = []
for _, labels in test_generator:
    true_classes.extend(np.argmax(labels, axis=1))  # Assuming labels are one-hot encoded
true_classes = np.array(true_classes)

# Calculate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Classes')
plt.ylabel('True Classes')
plt.show()

In [ ]:
# Load TensorBoard
%load_ext tensorboard

# Start TensorBoard
logdir = "logs/fit/"  # Ensure this matches your logging path
%tensorboard --logdir {logdir}